# Tweet Turing Test: Detecting Disinformation on Twitter  

|          | Group #2 - Disinformation Detectors                     |
|---------:|---------------------------------------------------------|
| Members  | John Johnson, Katy Matulay, Justin Minnion, Jared Rubin |
| Notebook | `01_merge.ipynb`                                        |
| Purpose  | Merge together acquired data into a common format.      |

Our acquired data is stored in the following directory structure:  
- `/` (root project directory)
    - `/data`
        - `/data/raw`
            - `/data/raw/tweets-govt_entities`
            - `/data/raw/tweets-inidviduals`
            - `/data/raw/tweets-news_orgs`
            - `/data/raw/tweets-random`
        - `/data/processed`
            - `/data/processed/...`

> (TODO - write more explaining notebook)

# Setup

In [3]:
# imports from Python standard library
import json
import logging
import os

# imports requiring installation
#   connection to Google Cloud Storage
from google.cloud import storage            # pip install google-cloud-storage
from google.oauth2 import service_account   # pip install google-auth

#  data science packages
import demoji                               # pip install demoji
import numpy as np                          # pip install numpy
import pandas as pd                         # pip install pandas

In [4]:
# imports from tweet_turing.py
from tweet_turing import get_files, merge_json_files, \
    get_gcp_storage_client, get_gcp_bucket, list_gcp_objects, get_gcp_object_as_json